In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.model_selection import train_test_split

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
train = pd.read_csv('./train.csv')

# pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test = pd.read_csv('./test.csv')
# pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

In [3]:
# ! pip install nltk

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /u/77/spilsbt1/unix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
english_stopwords = stopwords.words('english')

In [7]:
def remove_stop_word(sentence):
#     print(sentence)
    tokens = word_tokenize(sentence.lower())
    tokens_wo_stopwords = [t for t in tokens if t not in english_stopwords]
    return " ".join(tokens_wo_stopwords)

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/m/home/home7/77/spilsbt1/unix/english_feedback/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
test.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [10]:
train["full_text"] = train["full_text"].apply(remove_stop_word)
test["full_text"] = test["full_text"].apply(remove_stop_word)

In [11]:
train["full_text"][1]

"problem change let best matter happening change mind . sometimes need wake look around problems best way change want change along time ago . problem change make see different help understand tings wok . first make see different others . example remember came united states think nothing going change think nothing going change everything different country realist wrong problem may change sometimes change way , remember really shy think change lot sometimes problems make think thing never see life need see different way dont let nothing happened ruing change want make problem . example think nothing going change dont need shy anymore became need start seeing everything different ways get mad every one need know going happened , people may see different way know change best n't let nothing body change nothing . way want change one ca n't nothing choice problems decide . second help understand things work . instance mom lot problems faith around people , mom scare high 'm scare high unders

In [12]:
train[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]].describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
count,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000
mean,3.127077,3.028254,3.235745,3.116850,3.032856,3.081053
std,0.662542,0.644399,0.583148,0.655997,0.699841,0.671450
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [13]:
# add additional test data because we only have 3 test data point
training_data, additional_test  = train_test_split(train, train_size=0.95, random_state=1)

In [14]:
train_df, val_df = train_test_split(training_data, train_size=0.8, random_state=1)

In [15]:
sample_submission.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [16]:
train_x, train_y = np.array(train_df["full_text"]), np.array(train_df[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])
val_x, val_y = np.array(val_df["full_text"]), np.array(val_df[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])


In [17]:
train["full_text"]

0       think students would benefit learning home , w...
1       problem change let best matter happening chang...
2       dear , principal u change school policy grade ...
3       best time life become . agree greatest accompl...
4       small act kindness impact people change people...
                              ...                        
3906    believe using cellphones class education us go...
3907    working alone , students argue decission proye...
3908    `` problem chance best '' think quote cant bes...
3909    many people disagree albert schweitzer 's quot...
3910    think failure main thing people consist goal '...
Name: full_text, Length: 3911, dtype: object

In [18]:
test_x = np.array(test["full_text"])
test_y = np.array(sample_submission[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])

In [19]:
additional_test_x, additional_test_y = np.array(additional_test["full_text"]), np.array(additional_test[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])


In [20]:
# test_x

In [21]:
len(train_x)

2972

In [22]:
import pdb

In [23]:
def get_token_and_masks(sentence_arr):
#     pdb.set_trace()
#     print("meooowww",sentence_arr)
    tokens_arr = []
    tokens_type_ids_arr = []
    attention_mask = []
    for item in sentence_arr:
#         pdb.set_trace()
        tokens = tokenizer(item, padding="max_length", truncation=True, max_length=512)
        if len(tokens["input_ids"]) != 512:
            print("wtf",len(tokens["input_ids"]), tokens)
        tokens_arr.append(np.array(tokens["input_ids"]))
        tokens_type_ids_arr.append(np.array(tokens["token_type_ids"]))
        attention_mask.append(np.array(tokens["attention_mask"]))
    return np.array(tokens_arr), np.array(tokens_type_ids_arr), np.array(attention_mask)

In [24]:
train_tokens_arr, train_token_type_ids_arr, train_attention_mask = get_token_and_masks(train_x)
val_tokens_arr, val_token_type_ids_arr, val_attention_mask = get_token_and_masks(val_x)


In [25]:
test_tokens_arr, test_token_type_ids_arr, test_attention_mask = get_token_and_masks(test_x)

In [26]:
a_test_tokens_arr, a_test_token_type_ids_arr, a_test_attention_mask = get_token_and_masks(additional_test_x)

In [27]:
train_tokens_arr

array([[ 101, 1234, 1294, ...,    0,    0,    0],
       [ 101, 3776, 1278, ...,    0,    0,    0],
       [ 101, 1242, 1234, ...,    0,    0,    0],
       ...,
       [ 101, 1242, 1651, ...,    0,    0,    0],
       [ 101, 9848, 8341, ...,    0,    0,    0],
       [ 101,  169,  169, ...,    0,    0,    0]])

### DataLoader

In [28]:
train_dataset = list(zip(train_tokens_arr, train_token_type_ids_arr, train_attention_mask, train_y))
val_dataset = list(zip(val_tokens_arr, train_token_type_ids_arr, val_attention_mask, val_y))


In [29]:
test_dataset = list(zip(test_tokens_arr, test_token_type_ids_arr, test_attention_mask))

In [30]:
a_test_dataset = list(zip(a_test_tokens_arr, a_test_token_type_ids_arr, a_test_attention_mask))

In [31]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=4, drop_last=True)

In [32]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=4, drop_last=False)

In [33]:
a_test_dataset

[(array([  101,  1148, 25263,  1593,  4763,  1849,   119, 23423,  1440,
          1825,  1750,   127,  3071,  1341,  1221,  1825,   119,  3662,
          1891,  1825,  1655,  1995, 22532, 20311,   119,  8351,  1849,
           119,  1649, 23423,  1148, 25263,  1593,  4763,  1849,   119,
          2255,  1236,  1825,  2736,   117,  5935,   117,  4909,  4196,
          1579,  1849,  2006,  1553,  2458,  1825,   119,  1148,  8351,
           117,  1440,   119,  1649,  1236,  1825,  2736,  1148,  8351,
          2762,  1204,  1579,  1541,   119,  8310,  1800,   183,   112,
           189,  2276,  1193,   119,  1825,  1363,  1702,  1825,   112,
          1396,  1518,  1562,  1541,  1328,  2283,   119,   192, 15540,
          1825,   117,  4819,  1277,   119,  1145,  1759,  1236,  1213,
           117,  1825,  1180,   190, 23655,  2556,  1825,  1518,  1899,
           117,  1309,  1522,  2640,   119,  1115,  1116,  4663,   117,
          1825,  1541,  1363,   119,  1145,  1176,  1274,  1204,

In [34]:
additional_test_dataloader = DataLoader(a_test_dataset, shuffle=False, batch_size=16, drop_last=False)

In [35]:
# tokenizer(train_x[0], padding=True, truncation=True, max_length=128)["input_ids"]

In [36]:
# for token, ids, mask, label in train_dataloader:
#     print(token, ids, mask, label)
    

### Model

In [40]:
import torch.nn as nn

In [41]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [45]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [46]:
class ModelWrapper(nn.Module):
    def __init__(self, bert_module, n_outputs, scale):
        super().__init__()
        self.bert_module = bert_module
        self.linear = nn.Linear(768, 6)
        self.scale = scale
    
    def forward(self, input_ids, token_type_ids, attention_mask):
        pooler_output = self.bert_module(
            input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
        ).pooler_output
        return self.linear(pooler_output).sigmoid() * self.scale

In [47]:
wrapper = ModelWrapper(model.bert, 6, 5.5)

### Optimizer and learning rate scheduler

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps
)

In [ ]:
wrapper.to(device)

In [ ]:
from tqdm.auto import tqdm

def train():
    progress_bar = tqdm(range(num_training_steps))
    mse_loss = nn.MSELoss()

    wrapper.train()
    
    try:
        for epoch in range(num_epochs):
            for tokens, ids, masks, labels in train_dataloader:
#                 print("epoch", epoch)
                tokens = tokens.to(device)
                ids = ids.to(device)
                masks = masks.to(device)
                labels = labels.to(device)
                outputs = wrapper(tokens, ids, masks)

                loss = mse_loss(outputs, labels.float())
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                progress_bar.set_postfix({"loss": loss.item()})
    except KeyboardInterrupt:
        print("interrupt")
    torch.save(wrapper, "saved_model_rmsw.pt")
    return wrapper

wrapper = train()

In [42]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [48]:
model = torch.load("saved_model_rmsw.pt")
model.to(device)
model.eval()

ModelWrapper(
  (bert_module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [49]:
for tokens, ids, masks in test_dataloader:
    tokens = tokens.to(device)
    ids = ids.to(device)
    masks = masks.to(device)
    preds = wrapper(tokens, ids, masks)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

In [50]:
def evaluation(model, dataloader):
    with torch.inference_mode():
        evaluation_prediction_arr = []
        for tokens, ids, masks in dataloader:
            tokens = tokens.to(device)
            ids = ids.to(device)
            masks = masks.to(device)
            a_preds = model(tokens, ids, masks).cpu().detach().numpy()
            pred_transform = np.around(a_preds*2)/2
            evaluation_prediction_arr.append(pred_transform)
        return np.concatenate(evaluation_prediction_arr)

In [51]:
sample_submission.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [52]:
preds

NameError: name 'preds' is not defined

In [ ]:
evaluation(wrapper, test_dataloader)

In [53]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [54]:
additional_test_y.shape

(196, 6)

In [55]:
a = evaluation(model, additional_test_dataloader)

In [56]:
a

array([[3.5, 3.5, 3.5, 3.5, 3.5, 3. ],
       [3. , 3.5, 3. , 3.5, 3. , 3.5],
       [3. , 3.5, 3.5, 3. , 3.5, 3.5],
       ...,
       [2.5, 2.5, 3. , 3. , 2.5, 2.5],
       [3.5, 2.5, 3. , 3. , 3. , 3. ],
       [3.5, 3. , 3. , 3. , 2.5, 3.5]], dtype=float32)

In [57]:
mean_squared_error(additional_test_y, a)

0.31590136054421775